<a href="https://colab.research.google.com/github/kasier48/DeepLearning/blob/main/Pratice_Week5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install langchain-community langchain-chroma langchain-openai bs4

In [39]:
import os
import sys
import requests
import bs4
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# [MYCODE] 기존 WebBaseLoader를 이용해서 가져오니 한글 꺠짐 문제 현상으로 인하여 순수 api get으로 대체한다.
url = "https://spartacodingclub.kr/blog/all-in-challenge_winner"
response = requests.get(url)

# [MYCODE] 한글을 꺠지지 않기 위해서는 utf-8로 설정되어 있는지 확인한다.
print(f'설정된 인코딩: {response.encoding}')

# [MYCODE] BeautifulSoup으로 파싱하고 editedContent class을 찾는다.
soup = bs4.BeautifulSoup(response.text, 'html.parser')
content_elements = soup.find_all(class_='editedContent')

# [MYCODE] docs의 내용을 확인.
docs = [Document(page_content=element.get_text(separator='\n', strip=True)) for element in content_elements]
for idx, doc in enumerate(docs):
  print(f"idx; {idx + 1}, content: {doc.page_content}")

설정된 인코딩: utf-8
idx; 1, content: 코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.
<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.
🏆 대상
[Lexi Note] 언어공부 필기 웹 서비스
서비스 제작자: 다나와(김다애, 박나경)
💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.
W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 있습니다. 필기, 사전, 번역을 한번에 쉽고 편하게 이용할 수 있죠. 더 이상 시간 낭비 없이 효율적으로 어문학을 공부하며 학습 속도도 눈에 띄게 빨라질 수 있어요. 언어 공부의 복잡함을 단순하게 만들어주는 Lexi Note가 요

In [37]:
from langchain import hub
from google.colab import userdata

# [MYCODE] openai_key 설정
openai_key = userdata.get('openai_key')

llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_key)

# [MYCODE] docs의 내용들을 토큰화 및 chunk 처리한다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=openai_key)
)

retriever = vectorstore.as_retriever()

user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt")

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# [MYCODE] langchain으로 묶어서 prompt를 실행한다.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print(rag_chain.invoke(user_msg))

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ALL-in 코딩 공모전에서는 대학생들이 캠퍼스 내 문제를 해결하기 위한 다양한 혁신적인 프로젝트가 수상작으로 선정되었습니다. 주요 수상작으로는 언어 학습을 효율적으로 도와주는 'Lexi Note', 벌레 퇴치 서비스를 제공하는 '우리집 히어로즈', 실시간 소통을 위한 '에코 클래스룸' 등이 있습니다. 이 외에도 외국인과 내국인을 연결하는 'BLOTIE'와 동아리 정보 플랫폼 'Crewing' 등 다양한 서비스가 개발되었습니다.
